In [61]:
from gensim.parsing.preprocessing import stem_text
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim import corpora, models 
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
sys.path.append('../')
from utils import word_frequency_per_player, remove_similar_rows_per_player
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import pyLDAvis
from pyLDAvis.gensim_models import prepare
import re
from seaborn import countplot, distplot, heatmap, histplot
import numpy as np




# Load data and convert to stemmed and BERT data 

In [62]:
# load data and drop NA's
df = pd.read_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Preprocessing/data_clean/de_clean_3.csv')

# Because Bayer Leverkusen and Leverkusen appeared in every single cluster we remove it 

In [63]:
df['data'] = df['data'].apply(lambda x: re.sub(r"\b(bayerleverkusen)\b", '', str(x)))
df['data'] = df['data'].apply(lambda x: re.sub(r"\b(leverkusen)\b", '', str(x)))
df['data'] = df['data'].apply(lambda x: re.sub(r"\b(nicht)\b", '', str(x)))
df['data'] = df['data'].apply(lambda x: re.sub(r"\b(uber)\b", '', str(x)))

# Stemming and Vectorization

In [64]:
# create two copies 
df_stem = df.copy()

# create stemmed data
df_stem['data'] = df_stem['data'].apply(stem_text)

df_stem

,data,player,language,publishedAt
0,trainer alonso mitgereisten fan monaco glucksmoment hebt tor,exequiel palacios,de,2023-02-24T09:33:31Z
1,sechser droht viereinhalb monaten paus bitter abschi dafur erscheint verletzungspaus lang zudem konkurrenzsitu doppelsech seit woch voll saft stehenden amiri demirbai andrich voraussetzungen extrem schwierig aranguiz,exequiel palacios,de,2023-03-03T21:35:13Z
2,sechser erklart budapest spiel jungsten fortschritt donnerstag achtelfin hinspiel europaleagu ferencvaro budapest empfangt andrich fehlen sah defens mittelfeldspiel erfolg elfmeterschiessen monaco dritt gelb kart gesperrt massiv schwachung werkself doppelsech andrich zuletzt erfolgsfaktor,exequiel palacios,de,2023-03-07T11:34:39Z
3,bakker mittwoch abschluss train ausgeschlossen jedoch bakker ungarn startelf steht option erklart alonso mittelfeld doppel sech verzichten muss andrich gesperrt stehen demirbai amiri bereit,exequiel palacios,de,2023-03-08T14:25:18Z
4,aktuel nachrichten politik wirtschaft sport kultur allerd andrich argentinischen weltmeist beid eigentlich gesetzten sechser gesperrt charl aranguiz zudem verletzt donnerstag machten sach zumindest ordentlich,exequiel palacios,de,2023-03-09T19:53:46Z
...,...,...,...,...
617,wehrt europaleagu lang gut schwere hypothek ruckspiel romischen fan lautstark hymn choreo rang knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp,piero hincapie,de,2023-05-12T10:02:12Z
618,alonso taktik duell alten lehrmeist mourinho niederlag einstecken mussen droht einzug endspiel europaleagu verpassen halbfin hinspiel rom steht alonso trainiert werkself ruckspiel woch ganz schweren aufgab dritt europacup final vereinsgeschicht erreichen titel seit traumen romischen fan lautstark hymn choreo rang knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp,piero hincapie,de,2023-05-11T21:17:00Z
619,trotz pleit halbfin hinspiel rom gibt kampferisch luka hradecki knapp pleit kein katastroph zudem setzt kapitan hexenkessel eigenen stadion trainer alonso geht optimistisch ruckspiel romischen fan lautstark hymn choreo rang knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp,piero hincapie,de,2023-05-11T21:09:16Z
620,hitzigen aufeinandertreffen stadio olimpico nahm alonso alten lehrmeist arm erst taktik duell ging mourinho droht alonso einzug endspiel europaleagu verpassen romischen fan lautstark hymn choreo rang knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp,piero hincapie,de,2023-05-11T20:58:58Z


In [65]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=0.02)
df_tfidf = tfidf.fit_transform(df_stem.data).toarray()

In [66]:
df_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [67]:
#Store the frequency matrix in data_tfidf. 
data_tfidf = pd.DataFrame(df_tfidf, columns=tfidf.vocabulary_.keys())

#data_tfidf['target_names']=sorted_df_ng.target_names.values #we can't use that because otherwise knn wouldn't work
data_tfidf.head()

,trainer,alonso,fan,monaco,tor,sechser,droht,paus,dafur,lang,...,sehenswert,reih,freistehend,end,bedient,zog,winter,doppelpass,verwertet,lautstark
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.246436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
#Store the feature names in a words list.
words = data_tfidf.columns.tolist()[:-2] 
words

['trainer',
 'alonso',
 'fan',
 'monaco',
 'tor',
 'sechser',
 'droht',
 'paus',
 'dafur',
 'lang',
 'zudem',
 'seit',
 'woch',
 'amiri',
 'demirbai',
 'andrich',
 'erklart',
 'spiel',
 'donnerstag',
 'hinspiel',
 'europaleagu',
 'sah',
 'defens',
 'mittelfeldspiel',
 'erfolg',
 'dritt',
 'gelb',
 'kart',
 'gesperrt',
 'werkself',
 'zuletzt',
 'bakker',
 'jedoch',
 'startelf',
 'steht',
 'mittelfeld',
 'sech',
 'muss',
 'stehen',
 'bereit',
 'aktuel',
 'nachrichten',
 'politik',
 'wirtschaft',
 'sport',
 'kultur',
 'allerd',
 'weltmeist',
 'beid',
 'eigentlich',
 'verletzt',
 'zumindest',
 'union',
 'berlin',
 'chancen',
 'deutlich',
 'saint',
 'gillois',
 'alten',
 'erneut',
 'schwer',
 'kamen',
 'zuschauern',
 'minut',
 'gast',
 'fuhrung',
 'fussbal',
 'bundesligisten',
 'gut',
 'sein',
 'borussia',
 'dortmund',
 'entscheidenden',
 'moment',
 'gab',
 'bundesliga',
 'zuruck',
 'spielen',
 'weit',
 'geht',
 'auf',
 'platz',
 'sagt',
 'zweiten',
 'halft',
 'bisschen',
 'gefahrlich',
 's

In [69]:
kmeans = KMeans(n_clusters = 4, max_iter=1000, random_state=42)
kmeans.fit(data_tfidf)

/opt/homebrew/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(max_iter=1000, n_clusters=4, random_state=42)

In [70]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]

In [71]:
for num, centroid in enumerate(common_words):
    words_in_centroid = [words[word] for word in centroid if word < len(words)]
    print(f"{num}: {', '.join(words_in_centroid)}")
# By adding the if word < len(words) condition, we ensure that only valid indices are used to access the words list or dictionary.

0: klarheit, durchau, gelang, verdient, samstag, strafraum, weltmeist, gegeben, abstaub, duell
1: angriff, ebenfal, gespielt, ware, daniel, amiri, seit, fuhlt, pariert, millionen
2: erklart, ging, fussbal, innenverteidig, dagegen, schonem, europaleagu, wegen, dank, patricio
3: pariert, wurd, klub, seit, halbfinal, gelang, ruckspiel, spielt, steht, chanc


In [72]:
df_stem['cluster'] = kmeans.labels_
df_stem

,data,player,language,publishedAt,cluster
0,trainer alonso mitgereisten fan monaco glucksmoment hebt tor,exequiel palacios,de,2023-02-24T09:33:31Z,2
1,sechser droht viereinhalb monaten paus bitter abschi dafur erscheint verletzungspaus lang zudem konkurrenzsitu doppelsech seit woch voll saft stehenden amiri demirbai andrich voraussetzungen extrem schwierig aranguiz,exequiel palacios,de,2023-03-03T21:35:13Z,2
2,sechser erklart budapest spiel jungsten fortschritt donnerstag achtelfin hinspiel europaleagu ferencvaro budapest empfangt andrich fehlen sah defens mittelfeldspiel erfolg elfmeterschiessen monaco dritt gelb kart gesperrt massiv schwachung werkself doppelsech andrich zuletzt erfolgsfaktor,exequiel palacios,de,2023-03-07T11:34:39Z,2
3,bakker mittwoch abschluss train ausgeschlossen jedoch bakker ungarn startelf steht option erklart alonso mittelfeld doppel sech verzichten muss andrich gesperrt stehen demirbai amiri bereit,exequiel palacios,de,2023-03-08T14:25:18Z,2
4,aktuel nachrichten politik wirtschaft sport kultur allerd andrich argentinischen weltmeist beid eigentlich gesetzten sechser gesperrt charl aranguiz zudem verletzt donnerstag machten sach zumindest ordentlich,exequiel palacios,de,2023-03-09T19:53:46Z,1
...,...,...,...,...,...
617,wehrt europaleagu lang gut schwere hypothek ruckspiel romischen fan lautstark hymn choreo rang knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp,piero hincapie,de,2023-05-12T10:02:12Z,3
618,alonso taktik duell alten lehrmeist mourinho niederlag einstecken mussen droht einzug endspiel europaleagu verpassen halbfin hinspiel rom steht alonso trainiert werkself ruckspiel woch ganz schweren aufgab dritt europacup final vereinsgeschicht erreichen titel seit traumen romischen fan lautstark hymn choreo rang knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp,piero hincapie,de,2023-05-11T21:17:00Z,3
619,trotz pleit halbfin hinspiel rom gibt kampferisch luka hradecki knapp pleit kein katastroph zudem setzt kapitan hexenkessel eigenen stadion trainer alonso geht optimistisch ruckspiel romischen fan lautstark hymn choreo rang knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp,piero hincapie,de,2023-05-11T21:09:16Z,3
620,hitzigen aufeinandertreffen stadio olimpico nahm alonso alten lehrmeist arm erst taktik duell ging mourinho droht alonso einzug endspiel europaleagu verpassen romischen fan lautstark hymn choreo rang knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp,piero hincapie,de,2023-05-11T20:58:58Z,3


In [73]:
clusters = df_stem.groupby(['cluster']).size()
clusters

cluster
0     62
1     51
2    342
3    167
dtype: int64

# LDA Model

In [74]:
# Split the texts into a list of words
texts = [text.split() for text in df_stem['data']]

# Create the Gensim dictionary
dictionary = corpora.Dictionary(texts)

# Remove rare and common words from the dictionary
dictionary.filter_extremes(no_below=118, no_above=0.95)

# Derive the absolute frequency matrix using doc2bow
corpus = [dictionary.doc2bow(text) for text in texts]


In [75]:
lda = models.LdaModel(corpus, num_topics=4, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)


In [76]:
lda.show_topics()

[(0,
  '0.648*"alonso" + 0.349*"trainer" + 0.001*"wirtz" + 0.001*"leverkusen" + 0.001*"frimpong" + 0.001*"europaleagu" + 0.001*"spiel" + 0.001*"ball" + 0.001*"diabi"'),
 (1,
  '0.988*"spiel" + 0.002*"wirtz" + 0.002*"alonso" + 0.002*"leverkusen" + 0.002*"ball" + 0.001*"europaleagu" + 0.001*"trainer" + 0.001*"diabi" + 0.001*"frimpong"'),
 (2,
  '0.992*"europaleagu" + 0.002*"wirtz" + 0.001*"alonso" + 0.001*"leverkusen" + 0.001*"spiel" + 0.001*"frimpong" + 0.001*"trainer" + 0.001*"diabi" + 0.001*"ball"'),
 (3,
  '0.279*"wirtz" + 0.217*"frimpong" + 0.169*"leverkusen" + 0.168*"diabi" + 0.167*"ball" + 0.000*"alonso" + 0.000*"trainer" + 0.000*"europaleagu" + 0.000*"spiel"')]

In [77]:
topics=lda[corpus]
df_stem['lda_predicted_clusters']= [max(topics[i],key=lambda item:item[1])[0] for i in range(len(topics))]

In [78]:
df_stem.head()

,data,player,language,publishedAt,cluster,lda_predicted_clusters
0,trainer alonso mitgereisten fan monaco glucksmoment hebt tor,exequiel palacios,de,2023-02-24T09:33:31Z,2,0
1,sechser droht viereinhalb monaten paus bitter abschi dafur erscheint verletzungspaus lang zudem konkurrenzsitu doppelsech seit woch voll saft stehenden amiri demirbai andrich voraussetzungen extrem schwierig aranguiz,exequiel palacios,de,2023-03-03T21:35:13Z,2,0
2,sechser erklart budapest spiel jungsten fortschritt donnerstag achtelfin hinspiel europaleagu ferencvaro budapest empfangt andrich fehlen sah defens mittelfeldspiel erfolg elfmeterschiessen monaco dritt gelb kart gesperrt massiv schwachung werkself doppelsech andrich zuletzt erfolgsfaktor,exequiel palacios,de,2023-03-07T11:34:39Z,2,1
3,bakker mittwoch abschluss train ausgeschlossen jedoch bakker ungarn startelf steht option erklart alonso mittelfeld doppel sech verzichten muss andrich gesperrt stehen demirbai amiri bereit,exequiel palacios,de,2023-03-08T14:25:18Z,2,0
4,aktuel nachrichten politik wirtschaft sport kultur allerd andrich argentinischen weltmeist beid eigentlich gesetzten sechser gesperrt charl aranguiz zudem verletzt donnerstag machten sach zumindest ordentlich,exequiel palacios,de,2023-03-09T19:53:46Z,1,0


In [79]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(lda, corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.210280 -0.124870       1        1  48.573231
0     -0.410568 -0.000158       2        1  27.364029
1      0.078891 -0.259147       3        1  12.978414
2      0.121397  0.384174       4        1  11.084326, topic_info=          Term        Freq       Total Category  logprob  loglift
3        spiel  217.000000  217.000000  Default   9.0000   9.0000
2  europaleagu  186.000000  186.000000  Default   8.0000   8.0000
0       alonso  299.000000  299.000000  Default   7.0000   7.0000
1      trainer  161.000000  161.000000  Default   6.0000   6.0000
7        wirtz  229.000000  229.000000  Default   5.0000   5.0000
6     frimpong  178.000000  178.000000  Default   4.0000   4.0000
8   leverkusen  138.000000  138.000000  Default   3.0000   3.0000
5        diabi  138.000000  138.000000  Default   2.0000   2.0000
4         ball  137.000000  137.000000  Default   1.0000   1.0000
6     frimpong  177.926146  178.649992   Topic1  -1.5278   0.7180
7        wirtz  228.814307  229.821388   Topic1  -1.2763   0.7177
5        diabi  137.356275  138.073867   Topic1  -1.7866   0.7169
4         ball  136.640305  137.373340   Topic1  -1.7919   0.7167
8   leverkusen  138.240397  138.997199   Topic1  -1.7802   0.7166
1      trainer    0.236331  161.749137   Topic1  -8.1517  -5.8065
2  europaleagu    0.232762  186.315560   Topic1  -8.1669  -5.9631
3        spiel    0.232079  217.025270   Topic1  -8.1699  -6.1186
0       alonso    0.237509  299.994235   Topic1  -8.1468  -6.4192
0       alonso  299.216112  299.994235   Topic2  -0.4342   1.2933
1      trainer  161.020265  161.749137   Topic2  -1.0538   1.2914
8   leverkusen    0.238461  138.997199   Topic2  -7.5689  -5.0721
4         ball    0.232502  137.373340   Topic2  -7.5942  -5.0856
5        diabi    0.231663  138.073867   Topic2  -7.5978  -5.0943
6     frimpong    0.235443  178.649992   Topic2  -7.5816  -5.3358
2  europaleagu    0.233866  186.315560   Topic2  -7.5884  -5.3845
7        wirtz    0.262855  229.821388   Topic2  -7.4715  -5.4775
3        spiel    0.233636  217.025270   Topic2  -7.5893  -5.5381
3        spiel  216.378045  217.025270   Topic3  -0.0124   2.0389
4         ball    0.332382  137.373340   Topic3  -6.4909  -3.9823
8   leverkusen    0.334638  138.997199   Topic3  -6.4841  -3.9873
5        diabi    0.316793  138.073867   Topic3  -6.5389  -4.0354
1      trainer    0.319131  161.749137   Topic3  -6.5316  -4.1863
7        wirtz    0.411516  229.821388   Topic3  -6.2773  -4.2833
6     frimpong    0.314031  178.649992   Topic3  -6.5477  -4.3018
2  europaleagu    0.324111  186.315560   Topic3  -6.5161  -4.3122
0       alonso    0.344992  299.994235   Topic3  -6.4537  -4.7261
2  europaleagu  185.524822  186.315560   Topic4  -0.0085   2.1954
7        wirtz    0.332709  229.821388   Topic4  -6.3321  -4.3382
8   leverkusen    0.183703  138.997199   Topic4  -6.9261  -4.4292
5        diabi    0.169136  138.073867   Topic4  -7.0087  -4.5052
4         ball    0.168151  137.373340   Topic4  -7.0146  -4.5060
1      trainer    0.173410  161.749137   Topic4  -6.9838  -4.6385
6     frimpong    0.174372  178.649992   Topic4  -6.9782  -4.7324
3        spiel    0.181510  217.025270   Topic4  -6.9381  -4.8868
0       alonso    0.195622  299.994235   Topic4  -6.8632  -5.1357, token_table=      Topic      Freq         Term
term                              
0         2  0.996686       alonso
4         1  0.997282         ball
5         1  0.992223        diabi
2         4  0.998306  europaleagu
6         1  0.996362     frimpong
8         1  0.992826   leverkusen
3         3  0.995276        spiel
1         2  0.995369      trainer
7         1  0.996426        wirtz, R=9, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3])

## Create a second model

In [80]:
lda_2 = models.LdaModel(corpus, num_topics=10, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)

## Finding the best number of topics

In [81]:
models_arr = {'cluster_' + str(iter): '' for iter in range(2, 15)}
for iter in range(2, 15): 
    models_arr.update({'cluster_'+ str(iter): models.LdaModel(corpus, num_topics=iter, id2word=dictionary, chunksize=150, iterations=100, passes=20, random_state=42)})

In [106]:
models_arr['cluster_9'].show_topics(num_words = 15)

[(0,
  '0.880*"wirtz" + 0.115*"diabi" + 0.001*"ball" + 0.001*"alonso" + 0.001*"frimpong" + 0.001*"leverkusen" + 0.001*"trainer" + 0.001*"spiel" + 0.001*"europaleagu"'),
 (1,
  '0.994*"spiel" + 0.001*"alonso" + 0.001*"wirtz" + 0.001*"trainer" + 0.001*"ball" + 0.001*"europaleagu" + 0.001*"diabi" + 0.001*"frimpong" + 0.001*"leverkusen"'),
 (2,
  '0.794*"leverkusen" + 0.200*"diabi" + 0.001*"trainer" + 0.001*"alonso" + 0.001*"europaleagu" + 0.001*"wirtz" + 0.001*"ball" + 0.001*"frimpong" + 0.001*"spiel"'),
 (3,
  '0.761*"europaleagu" + 0.236*"wirtz" + 0.000*"alonso" + 0.000*"spiel" + 0.000*"trainer" + 0.000*"leverkusen" + 0.000*"diabi" + 0.000*"frimpong" + 0.000*"ball"'),
 (4,
  '0.357*"ball" + 0.340*"diabi" + 0.301*"frimpong" + 0.000*"wirtz" + 0.000*"alonso" + 0.000*"trainer" + 0.000*"leverkusen" + 0.000*"europaleagu" + 0.000*"spiel"'),
 (5,
  '0.393*"wirtz" + 0.309*"spiel" + 0.287*"leverkusen" + 0.002*"alonso" + 0.002*"frimpong" + 0.002*"diabi" + 0.002*"ball" + 0.002*"europaleagu" + 0.002

In [116]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(models_arr['cluster_4'], corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.265663  0.072279       1        1  29.359846
3      0.237058 -0.004185       2        1  27.534851
2     -0.198534 -0.368403       3        1  22.230179
1     -0.304188  0.300309       4        1  20.875124, topic_info=          Term        Freq       Total Category  logprob  loglift
0       alonso  250.000000  250.000000  Default   9.0000   9.0000
4         ball  186.000000  186.000000  Default   8.0000   8.0000
5        diabi  189.000000  189.000000  Default   7.0000   7.0000
6     frimpong  181.000000  181.000000  Default   6.0000   6.0000
3        spiel  165.000000  165.000000  Default   5.0000   5.0000
7        wirtz  196.000000  196.000000  Default   4.0000   4.0000
1      trainer  192.000000  192.000000  Default   3.0000   3.0000
8   leverkusen  152.000000  152.000000  Default   2.0000   2.0000
2  europaleagu  172.000000  172.000000  Default   1.0000   1.0000
0       alonso  250.243638  250.937386   Topic1  -0.6833   1.2228
1      trainer  191.828037  192.525947   Topic1  -0.9492   1.2219
2  europaleagu   52.065227  172.332377   Topic1  -2.2533   0.0286
8   leverkusen    0.244419  152.452703   Topic1  -7.6146  -5.2102
3        spiel    0.247023  165.282429   Topic1  -7.6040  -5.2804
6     frimpong    0.244118  181.633632   Topic1  -7.6159  -5.3866
4         ball    0.239665  186.624072   Topic1  -7.6343  -5.4321
5        diabi    0.238638  189.772074   Topic1  -7.6386  -5.4531
7        wirtz    0.243423  196.439412   Topic1  -7.6187  -5.4678
7        wirtz  195.708355  196.439412   Topic2  -0.8650   1.2860
8   leverkusen  148.031905  152.452703   Topic2  -1.1442   1.2603
2  europaleagu  119.781953  172.332377   Topic2  -1.3559   0.9260
3        spiel    0.214352  165.282429   Topic2  -7.6817  -5.3581
6     frimpong    0.213023  181.633632   Topic2  -7.6879  -5.4586
4         ball    0.208730  186.624072   Topic2  -7.7083  -5.5061
5        diabi    0.208173  189.772074   Topic2  -7.7110  -5.5255
1      trainer    0.210209  192.525947   Topic2  -7.7012  -5.5302
0       alonso    0.211592  250.937386   Topic2  -7.6947  -5.7886
5        diabi  189.065026  189.772074   Topic3  -0.6855   1.5000
6     frimpong  180.914184  181.633632   Topic3  -0.7296   1.4998
8   leverkusen    3.915718  152.452703   Topic3  -4.5626  -2.1581
3        spiel    0.217749  165.282429   Topic3  -7.4520  -5.1283
2  europaleagu    0.221159  172.332377   Topic3  -7.4365  -5.1546
4         ball    0.237386  186.624072   Topic3  -7.3656  -5.1634
1      trainer    0.226038  192.525947   Topic3  -7.4146  -5.2436
7        wirtz    0.226297  196.439412   Topic3  -7.4135  -5.2625
0       alonso    0.221879  250.937386   Topic3  -7.4332  -5.5271
4         ball  185.938292  186.624072   Topic4  -0.6393   1.5629
3        spiel  164.603305  165.282429   Topic4  -0.7611   1.5625
8   leverkusen    0.260661  152.452703   Topic4  -7.2092  -4.8048
2  europaleagu    0.264038  172.332377   Topic4  -7.1963  -4.9145
6     frimpong    0.262307  181.633632   Topic4  -7.2029  -4.9736
5        diabi    0.260237  189.772074   Topic4  -7.2109  -5.0254
1      trainer    0.261664  192.525947   Topic4  -7.2054  -5.0343
7        wirtz    0.261336  196.439412   Topic4  -7.2066  -5.0557
0       alonso    0.260277  250.937386   Topic4  -7.2107  -5.3046, token_table=      Topic      Freq         Term
term                              
0         1  0.996264       alonso
4         4  0.996656         ball
5         3  0.995932        diabi
2         1  0.301742  europaleagu
2         2  0.696329  europaleagu
6         3  0.996511     frimpong
8         2  0.970793   leverkusen
8         3  0.026238   leverkusen
3         4  0.998291        spiel
1         1  0.997268      trainer
7         2  0.997763        wirtz, R=9, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 2])

# Examining the model with 8 clusters

In [117]:
# Create an empty array to store the most likely topics
most_likely_topics = []

#loop over all documents within the corpus
for doc in corpus:
    
    # predict the topics for the documents using the model with 8 clusters
    topics = models_arr['cluster_4'].get_document_topics(doc)

    # Get the most likely topic for the document
    most_likely_topic = max(topics, key=lambda x: x[1])
    
    # Append the most likely topic ID to the array
    most_likely_topics.append(most_likely_topic[0])

# Print the array of most likely topics
print(most_likely_topics)


[0, 0, 1, 0, 0, 3, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 1, 3, 3, 1, 0, 3, 1, 1, 1, 1, 3, 0, 0, 3, 2, 3, 0, 1, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 1, 0, 1, 0, 3, 3, 1, 2, 3, 2, 0, 0, 3, 1, 3, 3, 0, 1, 1, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 3, 0, 1, 0, 1, 1, 1, 3, 0, 1, 1, 0, 1, 3, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 3, 0, 2, 3, 2, 2, 3, 3, 0, 0, 3, 0, 2, 0, 1, 0, 1, 0, 0, 0, 1, 3, 3, 2, 3, 0, 0, 3, 2, 3, 3, 2, 2, 0, 0, 0, 3, 1, 0, 1, 1, 0, 0, 0, 2, 2, 0, 3, 0, 2, 0, 0, 2, 0, 0, 2, 1, 0, 2, 2, 2, 3, 0, 0, 0, 0, 1, 1, 3, 0, 2, 0, 2, 0, 0, 2, 0, 1, 2, 3, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 1, 3, 1, 1, 0, 0, 1, 3, 2, 3, 0, 0, 1, 0, 0, 0, 0, 2, 2, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 1, 0, 2, 0, 0, 0, 3, 1, 3, 1, 2, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 2, 2, 2, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 3, 3, 3, 1, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 1, 

In [118]:
# create a new column to append the predicted clusters
df['predicted_cluster'] = most_likely_topics

# show countplot
df["predicted_cluster"].value_counts()


predicted_cluster
0    282
3    121
2    120
1     99
Name: count, dtype: int64

In [ ]:
# 0 = euro league
# 1 
# 2 bundesliga

In [119]:
df

,data,player,language,publishedAt,predicted_cluster,topic
0,trainer alonso mitgereisten fans monaco glucksmoment hebt tor,exequiel palacios,de,2023-02-24T09:33:31Z,0,Game description
1,sechser droht viereinhalb monaten pause bitterer abschied dafur erscheint verletzungspause lang zudem konkurrenzsituation doppelsechs seit woche voll saft stehenden amiri demirbay andrich voraussetzungen extrem schwierige aranguiz,exequiel palacios,de,2023-03-03T21:35:13Z,0,Europa League
2,sechser erklart budapest spiel jungsten fortschritt donnerstag achtelfinal hinspiel europaleague ferencvaros budapest empfangt andrich fehlen sah defensive mittelfeldspieler erfolg elfmeterschiessen monaco dritte gelbe karte gesperrt massive schwachung werkself doppelsechs andrich zuletzt erfolgsfaktor,exequiel palacios,de,2023-03-07T11:34:39Z,1,Other
3,bakker mittwoch abschluss training ausgeschlossen jedoch bakker ungarn startelf steht option erklart alonso mittelfeld doppel sechs verzichten muss andrich gesperrt stehen demirbay amiri bereit,exequiel palacios,de,2023-03-08T14:25:18Z,0,Game description
4,aktuelle nachrichten politik wirtschaft sport kultur allerdings andrich argentinischen weltmeister beide eigentlich gesetzten sechser gesperrt charles aranguiz zudem verletzt donnerstag machten sache zumindest ordentlich,exequiel palacios,de,2023-03-09T19:53:46Z,0,Europa League
...,...,...,...,...,...,...
617,wehrt europaleague lange gut schwere hypothek ruckspiel romischen fans lautstarker hymne choreo range knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigte unbeeindruckt beinahe blitzstart hingelegt sekunden gespielt andrich hereingabe etwa zwolf metern schuss kam zentral halbhoch sodass portugiesische torhuter rui patricio seit donnerstag einsatzen rekordspieler europaleague wenig muhe hatte chancenlos ware portugiesische europameister sechsten minute gewesen wirtz verzog knapp,piero hincapie,de,2023-05-12T10:02:12Z,3,Player Performance
618,alonso taktik duell alten lehrmeister mourinho niederlage einstecken mussen droht einzug endspiel europaleague verpassen halbfinal hinspiel rom steht alonso trainierte werkself ruckspiel woche ganz schweren aufgabe dritte europacup finale vereinsgeschichte erreichen titel seit traumen romischen fans lautstarker hymne choreo range knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigte unbeeindruckt beinahe blitzstart hingelegt sekunden gespielt andrich hereingabe etwa zwolf metern schuss kam zentral halbhoch sodass portugiesische torhuter rui patricio seit donnerstag einsatzen rekordspieler europaleague wenig muhe hatte chancenlos ware portugiesische europameister sechsten minute gewesen wirtz verzog knapp,piero hincapie,de,2023-05-11T21:17:00Z,0,Player Performance
619,trotz pleite halbfinal hinspiel rom gibt kampferisch lukas hradecky knappe pleite keine katastrophe zudem setzt kapitan hexenkessel eigenen stadion trainer alonso geht optimistisch ruckspiel romischen fans lautstarker hymne choreo range knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigte unbeeindruckt beinahe blitzstart hingelegt sekunden gespielt andrich hereingabe etwa zwolf metern schuss kam zentral halbhoch sodass portugiesische torhuter rui patricio seit donnerstag einsatzen rekordspieler europaleague wenig muhe hatte chancenlos ware portugiesische europameister sechsten minute gewesen wirtz verzog knapp,piero hincapie,de,2023-05-11T21:09:16Z,0,Game description
620,hitzigen aufeinandertreffen stadio olimpico nahm alonso alten lehrmeister arm erste taktik duell ging mourinho droht alonso einzug endspiel europaleague verpassen romischen fans lautstarker hymne choreo range knallkorpern kurz spiel fast einschuchternden rahmen gesorgt zeigte unbeeindruckt beinahe blitzstart hingelegt sekunden gespielt andrich hereingabe etwa zwolf metern schuss kam zentral halbhoch sodass portugiesische torhuter rui patricio seit donnerstag einsatzen rekordspieler europaleague wenig muhe hatte chancenlos ware portugiesische eur

In [126]:
df[(df["predicted_cluster"] == 3)].head(20)


,data,player,language,publishedAt,predicted_cluster,topic
5,spaten tor joker sven michel union berlin mitreissenden achtelfinal hinspiel europaleague unentschieden gerettet chancen weiterkommen deutlich verbessert berliner taten royale union saint gilloise heimischen stadion alten forsterei erneut schwer kamen zuschauern schossen victor boniface minute yorbe vertessen gaste dreimal fuhrung josip juranovic robin knoche michel erzielten jeweiligen ausgleichstore fussball bundesligisten comeback qualitaten bewies,exequiel palacios,de,2023-03-09T20:01:53Z,3,Player Performance
19,adli verspricht schiedsrichter trikot duplizitat ereignisse bayern zweiten halfte bot zweimal drang adli munchner strafraum ein kam fall sah zweimal gelbe karte leverkusener offensivspieler gelb rot platz flog lag var team soren storks meldete namlich feld schiedsrichter tobias stieler chance bekam ursprungliche entscheidung korrigieren gab jeweils elfmeter werkself beide sicher endstand verwandelte,exequiel palacios,de,2023-03-19T20:00:00Z,3,Bundesliga News
20,nix stieler lag zwei entscheidenden situationen maximal daneben zweimal referee hamburg liga sonntagsspiel bayern munchen machtig geirrt zweimal bezichtigte leverkusener adli schwalbe bayern strafraum zweimal verhangte vermeintlichen tauschungsversuch voller uberzeugung gelbe karte und zweimal kleinlaut zurucknehmen entschuldigung bitten foulelfmeter geben zweimal verwandelte wurde uberraschender sieg,exequiel palacios,de,2023-03-19T20:01:45Z,3,Bundesliga News
23,dass videobeweis beitragen wurde jahrelang scheintote spannung bundesliga wiederzubeleben wer hatt gedacht genau sonntag bayarena bestaunen schreiben die spielminute xatopspiel werkself xaund rekordmeister xaleverkusens xaamine adli fallt xaim strafraum xader munchner xaschiedsrichter tobias stieler wittert allerdings oscarreife schauspieleinlage zeigt fachgerecht gelb xaausgerechnet kolner keller beweist jedoch unschuld jahrigen xastieler nimmt entscheidung samt verwarnung zuruck spricht leverkusener strafstoss der xaargentinische weltmeister lasst nehmen trifft schreiben spielminute xatopspiel werkself xaund rekordmeister xaleverkusens xaamine adli fallt xaim strafraum munchner xaschiedsrichter tobias stieler wittert allerdings oscarreife schauspieleinlage zeigt fachgerecht gelb xaausgerechnet kolner keller beweist jedoch unschuld jahrigen xastieler nimmt entscheidung samt verwarnung zuruck spricht leverkusener strafstoss der xaargentinische weltmeister lasst nehmen trifft,exequiel palacios,de,2023-03-20T05:11:00Z,3,Bundesliga News
28,protest schlechter altern konnen ausgerechnet leverkusener fans wahrend partie bayern grosses banner aufschrift videobeweis abschaffen gezeigt grossen profiteure technischen hilfsmittel zweimal regelhuter kolner keller verantwortender position soren storks gemeldet zweimal korrigierte schiedsrichter tobias stieler entscheidung zurecht zugunsten,exequiel palacios,de,2023-03-20T09:29:34Z,3,Bundesliga News
31,schalke reis nimmt spieler ausdrucklich schutz rollenverteilung klar tom krauss pflicht alleiniger sechser wirtz beschatten wahrend alex kral andrich drexler kummern sollten,exequiel palacios,de,2023-04-03T08:12:20Z,3,Bundesliga News
33,lasst gegner schlecht aussehen macht mitspieler besser teenager wirtz zweiter rechts danach wirtz mehrfach knapp dran lupenreinen assist hattrick entweder verdaddelten mitspieler gute ideen kevin trapp lenkte kaum haltbaren ball adli neben pfosten wirtz gerne ubersehen wird schafft spiel kollegen sicherheit geben frankfurt fielen verteidiger edmond tapsoba tah auf erfolgreich frankfurter sturmer randal kolo muani konzentrierten torschutze adli sturmer zeigten inspiriert wirtz wirbel ihr xakonnen,exequiel palacios,de,2023-04-08T19:43:08Z,3,Bundesliga News
43,spielt saint gilles ruckspiel verstecken viertelfinal ruckspiel europaleague union saint gilloise mochte karten schauen lassen fragezeichen adli daher wahlweise gestrichen gesetzt unmittelbar abschlusstraining stand alon

# Cluster naming 
0 = europa league
___
1 = other
___
2 = Bundesliga infos
___
3 = Bundesliga infos
___
4 = international_football_news
___
5 = player_performance
___
6 = Game descriptions
___
7 = Eurpoa League


In [86]:
def map_cluster_topic(cluster):
    if cluster == 0 or cluster == 7:
        return 'Europa League'
    elif cluster == 1:
        return 'Other'
    elif cluster == 2 or cluster == 3:
        return 'Bundesliga News'
    elif cluster == 4:
        return 'International Football News'
    elif cluster == 5:
        return 'Player Performance'
    else:
        return 'Game description'

# Apply the function and create 'sentiment_tb_desc' column
df['topic'] = np.vectorize(map_cluster_topic)(df['predicted_cluster'])

In [87]:
df.to_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Modeling/clustered_data_ger_v1.csv', index=False)

In [88]:
pd.set_option('display.max_colwidth', None)
df.head(200)

,data,player,language,publishedAt,predicted_cluster,topic
0,trainer alonso mitgereisten fans monaco glucksmoment hebt tor,exequiel palacios,de,2023-02-24T09:33:31Z,6,Game description
1,sechser droht viereinhalb monaten pause bitterer abschied dafur erscheint verletzungspause lang zudem konkurrenzsituation doppelsechs seit woche voll saft stehenden amiri demirbay andrich voraussetzungen extrem schwierige aranguiz,exequiel palacios,de,2023-03-03T21:35:13Z,0,Europa League
2,sechser erklart budapest spiel jungsten fortschritt donnerstag achtelfinal hinspiel europaleague ferencvaros budapest empfangt andrich fehlen sah defensive mittelfeldspieler erfolg elfmeterschiessen monaco dritte gelbe karte gesperrt massive schwachung werkself doppelsechs andrich zuletzt erfolgsfaktor,exequiel palacios,de,2023-03-07T11:34:39Z,1,Other
3,bakker mittwoch abschluss training ausgeschlossen jedoch bakker ungarn startelf steht option erklart alonso mittelfeld doppel sechs verzichten muss andrich gesperrt stehen demirbay amiri bereit,exequiel palacios,de,2023-03-08T14:25:18Z,6,Game description
4,aktuelle nachrichten politik wirtschaft sport kultur allerdings andrich argentinischen weltmeister beide eigentlich gesetzten sechser gesperrt charles aranguiz zudem verletzt donnerstag machten sache zumindest ordentlich,exequiel palacios,de,2023-03-09T19:53:46Z,0,Europa League
...,...,...,...,...,...,...
195,wirtz bisschen gala form vergangenen wochen entfernt gute verhaltnisse beinahe euphorische laune uberzeugend gespielten viertelfinale herrschte furs erste verweht roma sah aus seien elf fantasie frische abhandengekommen einflussreichsten spieler wirken erschopft wirtz startete stark verblasste starker diaby lichtgeschwinde duett rechten seite traten kaum erscheinung hradecky mitbekommen hatte mussen verbessern sagte unuberhorbaren seufzen wir wussten leicht knacken sind mittel wege mussen finden gelingt droht schlimmste bestrafungen sportliche rauswurf wettbewerb boser stempel kompletten mission nachlassen drauf ankommt wurde altbekannte stigma klub vizekusen wachrufen nichts hassen mehr,jeremie frimpong,de,2023-05-12T10:25:02Z,2,Bundesliga News
196,wehrt europaleague lange gut schwere hypothek ruckspiel wechsel machte roma machtig druck muhe spiel beruhigen recht gelang prompt fiel edmond tapsoba andrich ball wegbekamen hradecky zunachst tammy abraham parierte bove nachschuss erfolgreich war kurz schluss kam grossen chance ausgleich roma verteidiger bryan cristante rettete brust kurz torlinie geschlagenen torhuter weniger sorgen verantwortlichen aussenbahnspieler halbzeit abbekommen schluss wieder sagte rolfes bei wahrscheinlich problematisch,jeremie frimpong,de,2023-05-12T10:02:12Z,1,Other
197,vergibt leichtfertig gute ausgangsposition werksklub fahrlassig zumindest leichtfertig gute ausgangsposition ruckspiel kommenden woche verschenkt die chancen machen urteilte rolfes durfte dabei direkt hundertprozentige schlussphase inkludieren niederlander torwartfehler rui patricio ball brust roma verteidiger bryan cristante platzierte statt verwaiste romer tor wir mussen besser spielen letzten drittel mahnte alonso zudem alten starken reaktivieren zuletzt spielen wolfsburg union berlin koln verschutt gegangen sind rechte seite lange schlussel erfolg derzeit stumpfe waffe wirkt uberspielt diaby bewegte roma ungefahrlichen raumen,jeremie frimpong,de,2023-05-12T09:44:10Z,4,International Football News
198,mourinho gewann roma duell schutzling alonso juventus rettete sevilla last minute tor edoardo bove erloste roma fans minute abraham schuss abpraller tor hradecky unterbrachte bemuhte schluss vergeblich ausgleich gefahrliche aktion pfiff schiedsrichter wegen fouls entstehung,jeremie frimpong,de,2023-05-12T08:15:00Z,6,Game description
